In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web
import scipy.optimize as opt
import math
import scipy.stats as stats
from scipy.stats import norm

In [6]:
def get_closes(tickers, start_date=None, end_date=None, freq=None):
    import pandas as pd
    pd.core.common.is_list_like = pd.api.types.is_list_like
    import pandas_datareader.data as web  
    closes = pd.DataFrame(columns = tickers, index=web.YahooDailyReader(symbols=tickers[0], start=start_date, end=end_date, interval=freq).read().index)
    for ticker in tickers:
        df = web.YahooDailyReader(symbols=ticker, start=start_date, end=end_date, interval=freq).read()
        closes[ticker]=df['Close']
    closes.index_name = 'Date'
    closes = closes.sort_index()
    return closes

### TICKERS -- FECHAS ###

In [19]:
# Información
ticker= ['SOXX', 'IYW', 'IHI', 'IBB', 'USMV'] 
start, end= '2018-10-18', '2019-10-18'

In [20]:
daily_closes = get_closes(ticker, start, end, freq='d') #d define es diario
daily_closes

,SOXX,IYW,IHI,IBB,USMV
Date,,,,,
2018-10-18,168.710007,180.100006,212.220001,113.000000,55.290001
2018-10-19,166.179993,179.460007,208.000000,111.220001,55.389999
2018-10-22,167.199997,180.929993,208.089996,109.620003,55.200001
2018-10-23,166.330002,180.470001,206.570007,109.089996,55.040001
2018-10-24,155.419998,171.779999,201.679993,102.809998,54.209999
2018-10-25,158.889999,177.589996,204.470001,103.989998,54.529999
2018-10-26,156.130005,173.660004,201.759995,103.610001,53.820000
2018-10-29,154.500000,170.410004,201.110001,102.610001,53.860001
2018-10-30,160.919998,173.199997,203.960007,103.629997,54.619999


In [21]:
daily_logret = np.log(daily_closes/daily_closes.shift()).dropna()
#daily_logret

### MÉTODO HISTÓRICO - 1 ACTIVO ###

In [22]:
## METODO HISTORICO #####
NC = 95
NC_LARGO = 100-NC
NC_CORTO = NC
VAR_HIST = pd.DataFrame(index=['VAR LARGO', 'VAR CORTO'], columns=ticker)

for i in range(len(ticker)):
    VAR_HIST.loc['VAR LARGO'][i]=np.percentile(daily_logret.iloc[:,i].values, NC_LARGO).round(6)*100
    VAR_HIST.loc['VAR CORTO'][i]=np.percentile(daily_logret.iloc[:,i].values, NC_CORTO).round(6)*100

VAR_HIST

,SOXX,IYW,IHI,IBB,USMV
VAR LARGO,-3.1346,-2.5905,-2.5616,-2.3949,-1.3398
VAR CORTO,2.9567,2.2006,1.7261,2.0462,1.0746


## MÉTODO PARAMÉTRICO - 1 ACTIVO ###

In [24]:
## METODO PARAMÉTRICO ####
NC = .95
NC_LARGO = 1-NC
NC_CORTO = NC
Z_LARGO= norm.ppf(NC_LARGO)
Z_CORTO= norm.ppf(NC_CORTO)
print("Z-VALUE Largo: ", Z_LARGO)
print("Z-VALUE Corto: ", Z_CORTO)


start, end= '2019-09-20', '2019-10-18'
daily_closes_lastmonth = get_closes(ticker, start, end, freq='d') #d define es diario
daily_logret_lastmonth = np.log(daily_closes_lastmonth/daily_closes_lastmonth.shift()).dropna()

##POSIBLE OPTIMIZACIÓN??, RESTAR A LA FECHA DEL PRINCIPIO??######
STDV = pd.DataFrame(index=['STDV'], columns=ticker)
for i in range(len(ticker)):
    STDV.loc['STDV'][i]=daily_logret_lastmonth.iloc[:,i].values.std().round(6)*100
STDV

VAR_PARAM = pd.DataFrame(index=['STDV', 'VAR LARGO', 'VAR CORTO'], columns=ticker)
for i in range(len(ticker)):
    VAR_PARAM.loc['STDV'][i]= float(STDV.iloc[:,i].values)
    VAR_PARAM.loc['VAR LARGO'][i]= float(STDV.iloc[:,i].values*Z_LARGO)
    VAR_PARAM.loc['VAR CORTO'][i]= float(STDV.iloc[:,i].values*Z_CORTO)
VAR_PARAM

Z-VALUE Largo:  -1.6448536269514722
Z-VALUE Corto:  1.6448536269514722


,SOXX,IYW,IHI,IBB,USMV
STDV,1.595,1.1654,1.2778,1.1534,0.6535
VAR LARGO,-2.62354,-1.91691,-2.10179,-1.89717,-1.07491
VAR CORTO,2.62354,1.91691,2.10179,1.89717,1.07491


### MÉTODO HISTÓRICO - PORTAFOLIO ###

In [36]:
PONDS = [ 0.23, 0.27, 0.21, 0.08, 0.21 ]
NC = 95
NC_LARGO = 100-NC
NC_CORTO = NC

SUMAPRODUCTO = pd.DataFrame(columns=['SUMAPRODUCTO'])
for i in range(len(daily_logret)):
    SUMAPRODUCTO.loc[i]= np.sum(daily_logret.iloc[i,:].values*np.transpose(PONDS))
#SUMAPRODUCTO

VAR_PORTA_HIST = pd.DataFrame(index=['VAR LARGO', 'VAR CORTO'], columns= ['VAR HIST PRTFLO'])
VAR_PORTA_HIST.loc['VAR LARGO']= np.percentile(SUMAPRODUCTO.iloc[:,].values, NC_LARGO).round(6)
VAR_PORTA_HIST.loc['VAR CORTO']= np.percentile(SUMAPRODUCTO.iloc[:,].values, NC_CORTO).round(6)
VAR_PORTA_HIST

,VAR HIST PRTFLO
VAR LARGO,-0.022946
VAR CORTO,0.018623


### MÉTODO PARAMÉTRICO - PORTAFOLIO ###

In [33]:
### METODO PARAMÉTRICO ####
PONDS =[ 0.23, 0.27, 0.21, 0.08, 0.21 ]
NC = .95
NC_LARGO = 1-NC
NC_CORTO = NC
Z_LARGO= norm.ppf(NC_LARGO)
Z_CORTO= norm.ppf(NC_CORTO)
print("Z-VALUE Largo: ", Z_LARGO)
print("Z-VALUE Corto: ", Z_CORTO)

start, end= '2019-09-20', '2019-10-18' ## 28 DIAS AL PASADO!
#https://www.timeanddate.com/date/workdays.html?d1=18&m1=10&y1=2019&d2=10&m2=9&y2=2019&ti=on&

daily_closes_lastmonth = get_closes(ticker, start, end, freq='d') #d define es diario
daily_logret_lastmonth = np.log(daily_closes_lastmonth/daily_closes_lastmonth.shift()).dropna()

Z-VALUE Largo:  -1.6448536269514722
Z-VALUE Corto:  1.6448536269514722


In [34]:
# MATRIZ DE COVARIANZA CON RENDIMIENTOS DIARIOS
cov_matrix = daily_logret_lastmonth.cov()
VARIANZA = cov_matrix.dot(PONDS).dot(np.transpose(PONDS))
cov_matrix


,SOXX,IYW,IHI,IBB,USMV
SOXX,0.000268,0.000189,0.000173,0.000141,0.000083
IYW,0.000189,0.000143,0.000131,0.000105,0.000067
IHI,0.000173,0.000131,0.000172,0.000112,0.000080
IBB,0.000141,0.000105,0.000112,0.000140,0.000048
USMV,0.000083,0.000067,0.000080,0.000048,0.000045


In [35]:
VAR_PARAM_PORT = pd.DataFrame(index=['VARIANZA', 'STDV', 'VAR LARGO', 'VAR CORTO'], columns= ['PARAM PRTFLO'])
VAR_PARAM_PORT.loc['VARIANZA']= VARIANZA
VAR_PARAM_PORT.loc['STDV']= np.sqrt(VARIANZA)
VAR_PARAM_PORT.loc['VAR LARGO']= np.sqrt(VARIANZA) * Z_LARGO
VAR_PARAM_PORT.loc['VAR CORTO']= np.sqrt(VARIANZA) * Z_CORTO
VAR_PARAM_PORT

,PARAM PRTFLO
VARIANZA,0.000127859
STDV,0.0113075
VAR LARGO,-0.0185991
VAR CORTO,0.0185991


### VAR - 1 DERIVADO ###

In [689]:
def VAR_DER(TITULOS, PRIMA, K, Z, SIGMASUB,DELTA):
    EXPOSURE = TITULOS*PRIMA
    return EXPOSURE*DELTA*SIGMASUB*Z


### PARÁMETROS DE LA OPCIÓN #### 
TITULOS = 5000000
K = 20
PRIMA = .362
DELTA = .48
SIGMASUB= .0051
NC = .05

####
Z = norm.ppf(NC)
VARDER = VAR_DER(TITULOS,PRIMA,K,Z,SIGMASUB,DELTA)
####
print("VAR DE OPCIÓN", VARDER.round(4))
print("PARA HACER NEUTRA MI POSICIÓN DEBO CUBRIR DELTA HEDGE CON:", DELTA*TITULOS)

VAR DE OPCIÓN -7288.149
PARA HACER NEUTRA MI POSICIÓN DEBO CUBRIR DELTA HEDGE CON: 2400000.0


In [728]:
### COBERTURA ###
SPOT = 19.444
SPOT_CHANGE = .20
DELTA_NEW = .4664
DELTAHEDGE = -TITULOS*DELTA
DELTAHEDGE_SPOT = DELTAHEDGE*SPOT
#### EN DOLARES
DELTAHEDGE, DELTAHEDGE_SPOT

(-2400000.0, -46665600.0)

In [726]:
PL = pd.DataFrame(index=['VALOR OPCION', 'VALOR SPOT'], columns= ['INICIALES', 'SPOT +-', 'TOTAL'])
PL.loc['VALOR OPCION']['INICIALES']=EXPOSURE
PL.loc['VALOR SPOT']['INICIALES']=DELTAHEDGE*SPOT
PL.loc['VALOR OPCION']['SPOT +-']=TITULOS*DELTA_NEW
PL.loc['VALOR SPOT']['SPOT +-']=DELTAHEDGE*(SPOT+SPOT_CHANGE)
PL.loc['VALOR OPCION']['TOTAL']=TITULOS*DELTA_NEW - EXPOSURE
PL.loc['VALOR SPOT']['TOTAL']=DELTAHEDGE*(SPOT+SPOT_CHANGE)-DELTAHEDGE*SPOT 
print("TU P&L NETO ES:", PL.loc['VALOR OPCION']['TOTAL']+PL.loc['VALOR SPOT']['TOTAL'])
####
PL

TU P&L NETO ES: 42000.00000000745


,INICIALES,SPOT +-,TOTAL
VALOR OPCION,1.81e+06,2.332e+06,522000
VALOR SPOT,-4.66656e+07,-4.71456e+07,-480000


### VAR - PORTAFOLIO  DERIVADOS ###